# Assignment - Deep Q-Learning

### Import libraries and relevant files

In [ ]:
## IMPORT
import numpy as np
import matplotlib.pyplot as plt
import sys

size_board = 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/UZH/Introduction to Reinforcement Learning/Chessboard/')

Mounted at /content/drive


In [ ]:
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     self.Board: A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X: The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a: The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R: The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done: A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [ ]:
## INITIALISE THE ENVIRONMENT
env = Chess_Env(size_board)

S, X, allowed_a = env.Initialise_game()                       # INTIALISE GAME

### Neural network training

In [ ]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 

S, X, allowed_a = env.Initialise_game()
N_a = np.shape(allowed_a)[0]                # TOTAL NUMBER OF POSSIBLE ACTIONS (= 32)

N_in = np.shape(X)[0]               ## INPUT SIZE (= 58)
N_out = 32                          ## OUTPUT SIZE
N_h = 200                           ## NUMBER OF HIDDEN NODES (= 200)

W1 = np.random.randn(N_h, N_in) * np.sqrt(1 / (N_in))
W2 = np.random.randn(N_out, N_h) * np.sqrt(1 / (N_h))
bias_W1 = np.zeros((N_h, ))
bias_W2 = np.zeros((N_out, ))


## INITALISE THE NEURAL NETWORK

def train(X, y, N_moves, W1, W2, bias_W1, bias_W2):     ## EPOCH = 1 (BATCH SIZE = SAMPLE SIZE)
    # INITIALISE THE GRADIENTS FOR EACH BATCH
    dW1 = np.zeros(W1.shape)
    dW2 = np.zeros(W2.shape)
    dbias_W1 = np.zeros(bias_W1.shape)
    dbias_W2 = np.zeros(bias_W2.shape)

    n_samples = X.shape[0]
    batch_size = N_moves
    shuffled_idxs = np.random.permutation(n_samples)

    for batch in range(0, batch_size):

        idx = shuffled_idxs[batch]
        x0 = X[idx]
        desired_output = y[idx]

        h1 = np.dot(W1, x0) + bias_W1       # NEURAL ACTIVATION: INPUT LAYER -> HIDDEN LAYER

        x1 = 1 / (1+np.exp(-h1))           # SIGMOID FUNCTION

        h2 = np.dot(W2, x1) + bias_W2      # NEURAL ACTIVATION: HIDDEN LAYER -> OUTPUT LAYER

        x2 = 1 / (1+np.exp(-h2))           # SIGMOID FUNCTION

        e_n = desired_output - x2          # COMPUTE THE ERROR SIGNAL

        delta2 = x2*(1-x2) * e_n           # BACKPROPAGATION: OUTPUT LAYER -> HIDDEN LAYER
        dW2 += np.outer(delta2, x1) 
        dbias_W2 += delta2

        delta1 = x1*(1-x1) * np.dot(W2.T, delta2)   # BACKPROPAGATION: HIDDEN LAYER -> INPUT LAYER
        dW1 += np.outer(delta1, x0)
        dbias_W1 += delta1

    # UPDATE THE WEIGHTS
    W2 += eta*dW2/batch_size
    W1 += eta*dW1/batch_size

    bias_W1 += eta*dbias_W1/batch_size
    bias_W2 += eta*dbias_W2/batch_size

    return W1, W2, bias_W1, bias_W2


## PREDICTION FROM THE NEURAL NETWORK

def predict(X, W1, W2, bias_W1, bias_W2):
    h1 = np.dot(W1, X) + bias_W1       # NEURAL ACTIVATION: INPUT LAYER -> HIDDEN LAYER

    x1 = 1 / (1+np.exp(-h1))           # SIGMOID FUNCTION

    h2 = np.dot(W2, x1) + bias_W2      # NEURAL ACTIVATION: HIDDEN LAYER -> OUTPUT LAYER

    x2 = 1 / (1+np.exp(-h2))           # SIGMOID FUNCTION

    return x2

### Epsilon-Greedy Policy

In [ ]:
## EPSILON-GREEDY POLICY

def EpsilonGreedy_Policy(q_val, epsilon):
    
    N_a = np.shape(q_val)[0]

    rand_value = np.random.uniform(0, 1)    # GENEATE A RANDOM NUMBER FROM THE UNIFORM DIST.

    rand_a = rand_value < epsilon

    if rand_a == True:
        a = np.random.randint(0, N_a)   # SELECTED ACTION (EXPLORE)
    else:
        a = np.argmax(q_val)            # SELECTED ACTION (EXPLOIT)
    
    return a

### Other Functions

In [ ]:
## OTHER FUNCTIONS

def checkAllow(allowed_a, q_val):               ## RETURNS THE ALLOWED ACTION INDEX
                                                ## AND THE CORRESPONDING Q VALUE
    q_val_dict = dict()
    for i in range(len(allowed_a)):
        if allowed_a[i][0] == 1:                # EQUALS 1 IF THE ACTION a[i] IS ALLOWED
            q_val_dict[i] = q_val[i]
    q_val_lst = list(q_val_dict.values())       # THE VALUE LISTS OF THE DICTIONARY (CORRESPONDING Q VALUE)
    allowed_lst = list(q_val_dict.keys())       # THE KEY LISTS OF THE DICTIONARY (ALLOWED ACTION INDEX)
    return q_val_dict, q_val_lst, allowed_lst

### Parameters

In [ ]:
# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2      # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.0005        # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85         # THE DISCOUNT FACTOR
eta = 0.1            # THE LEARNING RATE

N_episodes = 80000   # THE NUMBER OF GAMES TO BE PLAYED 
                     # 80000 FOR DQL AND 120000 FOR SARSA TABULAR

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])          # SAVE THE FINAL REWARD IN AN EPISODE
N_moves_save = np.zeros([N_episodes, 1])    # NUMBER OF MOVES

### Main Function

In [ ]:
# TRAINING LOOP BONE STRUCTURE
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...
np.random.seed(919)                             ## SET SEED

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)      ## DECAYING EPSILON
    Done = 0                                    ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    total_R = 0                                 ## TOTAL REWARD PER EPISODE
    i = 1                                       ## COUNTER FOR NUMBER OF ACTIONS

    # EPISODE DATA STORAGE
    q_val_episode = []                          ## Q values of an episode 
    X_episode = []                              ## X of an episode 
    
    S, X, allowed_a = env.Initialise_game()  
    q_val = predict(X, W1, W2, bias_W1, bias_W2)               ## Q-VALUE
    q_val_dict, q_val_lst, allowed_lst = checkAllow(allowed_a, q_val)
    if n % 5000 == 0:
        print(n)                                               ## CHECK THAT IT IS RUNNING
    
    while Done == 0:         ## START THE EPISODE

        a = EpsilonGreedy_Policy(q_val_lst, epsilon_f)                            # EPSILON-GREEDY POLICY
        S_next, X_next, allowed_a_next, R, Done = env.OneStep(allowed_lst[a])     # TAKE THE SELECTED ACTION

        total_R += R        # UPDATE TOTAL REWARD
        
        ## THE EPISODE HAS ENDED, UPDATE
        if Done == 1:
            q_val[allowed_lst[a]] = R       # SET THE Q-VALUE TO REWARD
            q_val_episode.append(q_val)
            X_episode.append(X)
            q_val_train = np.array(q_val_episode)   # A LIST OF Q VALUES IN AN EPISODE FOR WEIGHT UPDATE
            X_train = np.array(X_episode)           # A LIST OF X IN AN EPISODE FOR WEIGHT UPDATE
            W1, W2, bias_W1, bias_W2 = train(X_train, q_val_train, i, W1, W2, bias_W1, bias_W2)     # UPDATE THE WEIGHTS
            break
        

        ## IF THE EPISODE IS NOT OVER
        else:
            q_val_next = predict(X_next, W1, W2, bias_W1, bias_W2)          # PREDICT Q VALUE FOR NEXT ACTION
            delta = R + gamma*max(q_val_next) - q_val[allowed_lst[a]]
            q_val[allowed_lst[a]] = q_val[allowed_lst[a]] + eta * delta     # Q-VALUE UPDATE
            q_val_episode.append(q_val)     # COLLECT Q-VALUES FOR TRAINING
            X_episode.append(X)             # COLLECT X FOR TRAINING
            
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE  
        S = np.copy(S_next)
        X = np.copy(X_next)
        allowed_a = np.copy(allowed_a_next)
        q_val_dict, q_val_lst, allowed_lst = checkAllow(allowed_a, q_val)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

    R_save[n] = total_R
    N_moves_save[n] = i


### Saving R_save and N_moves_save as csv

In [ ]:
from numpy import savetxt
savetxt('/content/drive/MyDrive/UZH/Introduction to Reinforcement Learning/Chessboard/csv_files/table_R_q4.csv', R_save, delimiter=',')
savetxt('/content/drive/MyDrive/UZH/Introduction to Reinforcement Learning/Chessboard/csv_files/table_N_q4.csv', N_moves_save, delimiter=',')